In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Musiker e-shop: e-commerce trend prediction

### Author: Martin Schön (2024)

**Raw dataset under non-disclosure agreement (NDA) with https://www.muziker.sk/**

## Setup

In [ ]:
# !pip install wandb

In [ ]:
import tensorflow as tf

tf.get_logger().setLevel('FATAL')

import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, LSTM, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
matplotlib.style.use('ggplot')

import seaborn as sns
sns.set(context='paper', style='whitegrid', color_codes=True)   
sns.set_palette(sns.color_palette(["#017b92", "#f97306", "#ff0000"]))  # ["green", "orange", "red"] 

### Anonymized data for sale trend prediction made from NDA raw data

In [ ]:
ds = pd.read_csv("data/musiker-orders_days.csv",index_col=False, header=0)
daily_sales = ds.orders_amount
daily_sales.index = ds.order_created_at
daily_sales

In [ ]:
plt.figure(figsize=(20, 6))
daily_sales.plot(label='Daily Sales')
plt.title('Daily sales')
plt.xlabel('Date')
plt.ylabel('Number of sales')
plt.legend()
plt.show()

In [ ]:
# get all targets y from a TimeseriesGenerator instance.
def get_y_from_generator(gen):
    y = None
    for i in range(len(gen)):
        batch_y = gen[i][1]
        if y is None:
            y = batch_y
        else:
            y = np.append(y, batch_y)
    y = y.reshape((-1,1))
    print(y.shape)
    return y

In [ ]:
y = daily_sales.values
x = np.arange(0, y.size, 1)

data = y.reshape(-1,1)
print(data.shape)
plt.rcParams["figure.figsize"] = (20,4)
plt.plot(data)
plt.show()

# Clean data

In [ ]:
dftest = adfuller(data, autolag = 'AIC')
print("\t1. ADF : ",dftest[0])
print("\t2. P-Value : ", dftest[1])
print("\t3. Num Of Lags : ", dftest[2])
          
result = STL(data, period=6, robust = True).fit()
plt.rcParams["figure.figsize"] = (12,8)
result.plot()
plt.show()

# data_cleaned = result.trend 
# data_cleaned = result.trend + result.seasonal 
data_cleaned = result.trend + result.seasonal + result.resid     # all data without cleaning

data_cleaned = data_cleaned.flatten().reshape(-1, 1)

In [ ]:
# Plot all columns in 'data_cleaned'
plt.figure(figsize=(24, 6))
plt.plot(y)
plt.plot(data_cleaned)
plt.title('Plot of Numpy ndarray')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend(['Actual data', 'Data cleaned'])
plt.show()

# Training

In [ ]:
import os
import wandb
import logging
# Create logging 
logging.basicConfig(filename="training_log.log",
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

os.environ["WANDB_SILENT"] = "true"   # silence WANDB init as it gets a bit annoying with bigger trainings

wandb.login()

### Hyper-parameter tuning with wandb (Weights & Biases)

In [ ]:
from tensorflow.keras.callbacks import Callback

class WandbCallback(Callback):
    def __init__(self, config):
        super().__init__()
        self.config = config

    def on_epoch_end(self, epoch, logs=None):
        wandb.log(logs, step=epoch)


In [ ]:
def hyperparameter_tune(config: dict):
    run_name = f"train-lr{config['lr']}-ep{config['epochs']}-bs{config['batch_size']}-lu{config['lstm_units']}-lb{config['lookback']}-oa{config['output_activation']}"
    wandb.init(
            project="dp_lstm",
            config=config,
            name=run_name
        )
    
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_trans = scaler.fit_transform(data_cleaned)

    train_size = int(len(data_trans) * 0.80)
    test_size = len(data_trans) - train_size
    train, test = data_trans[0:train_size,:], data_trans[train_size:len(data_trans),:]

    look_back = config["lookback"]
    train_data_gen = TimeseriesGenerator(train, 
                                         train,
                                         length=look_back, 
                                         sampling_rate=1,
                                         stride=1,
                                         batch_size=config["batch_size"]
                                        )
    test_data_gen = TimeseriesGenerator(test, 
                                        test,
                                        length=look_back, 
                                        sampling_rate=1,
                                        stride=1,
                                        batch_size=config["batch_size"]
                                       )

    # model
    x = Input(shape=(look_back, 1))
    h = LSTM(units=config["lstm_units"])(x)   
    y = Dense(units=1, activation=config["output_activation"])(h)
    model = Model(inputs=x, outputs=y)
    #print(model.summary())
    
    # compile model
    opt = Adam(learning_rate=config['lr'])
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse', 'mae'])
    
    wandb_callback = WandbCallback(config)
    model.fit(train_data_gen, epochs=config["epochs"], shuffle=True, callbacks=[wandb_callback])
    
    return model.evaluate(test_data_gen)

In [ ]:
# WANDB HYPERPARAMETER TUNE, change the next line to True to use it 
wandb_tuning = False

config = {"lr": [0.1, 0.01, 0.001],
          "epochs": [5, 10, 20, 30, 50],
          "batch_size": [1, 3, 5, 8],
          "lstm_units": [3, 5, 7, 10],
          "lookback": [7, 10, 14],
          "output_activation": ['linear', 'sigmoid']
         }

if wandb_tuning:
    %%capture --no-stdout
    test_config = {}
    
    for lr in config["lr"]:
        test_config["lr"] = lr
        for epch in config["epochs"]:
            test_config["epochs"] = epch
            for bs in config["batch_size"]:
                test_config["batch_size"] = bs
                for lstm_u in config["lstm_units"]:
                    test_config["lstm_units"] = lstm_u
                    for lckb in config["lookback"]:
                        test_config["lookback"] = lckb
                        for outa in config["output_activation"]:
                            test_config["output_activation"] = outa

                            logging.info(f"Training - {test_config}")
                            res = hyperparameter_tune(test_config)
                            logging.info(f"Results - (loss, mse, mae): {res}")

In [ ]:
# BEST ACHIEVED RESULT 
BR_config = {'lr': 0.01, 'epochs': 30, 'batch_size': 1, 'lstm_units': 5, 'lookback': 10, 'output_activation': 'linear'}

# Visualize best results for sale trend prediction

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data_trans = scaler.fit_transform(data_cleaned)

train_size = int(len(data_trans) * 0.80)
test_size = len(data_trans) - train_size
train, test = data_trans[0:train_size,:], data_trans[train_size:len(data_trans),:]

look_back = BR_config["lookback"]
train_data_gen = TimeseriesGenerator(train, 
                                     train,
                                     length=look_back, 
                                     sampling_rate=1,
                                     stride=1,
                                     batch_size=BR_config["batch_size"]
                                    )
test_data_gen = TimeseriesGenerator(test, 
                                    test,
                                    length=look_back, 
                                    sampling_rate=1,
                                    stride=1,
                                    batch_size=BR_config["batch_size"]
                                   )

x = Input(shape=(look_back, 1))
h = LSTM(units=BR_config["lstm_units"])(x) 
y = Dense(units=1, activation=BR_config["output_activation"])(h)

model = Model(inputs=x, outputs=y)
print(model.summary())

# compile model
opt = Adam(learning_rate=BR_config['lr'])
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse', 'mae'])
model.fit(train_data_gen, epochs=BR_config["epochs"], shuffle=True)

In [ ]:
model.evaluate(test_data_gen)

In [ ]:
trainPredict = model.predict(train_data_gen)
print(trainPredict.shape)
testPredict = model.predict(test_data_gen)
print(testPredict.shape)

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)

In [ ]:
trainY = get_y_from_generator(train_data_gen)
testY = get_y_from_generator(test_data_gen)

In [ ]:
trainY = scaler.inverse_transform(trainY)
testY = scaler.inverse_transform(testY)

In [ ]:
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:, 0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
begin =  look_back
end = begin + len(trainPredict)

trainYPlot = np.empty_like(data, dtype=float)  # Ensure dtype is float
trainYPlot[:, :] = np.nan
trainYPlot[begin:end, :] = trainY

trainPredictPlot = np.empty_like(data, dtype=float)  # Ensure dtype is float
trainPredictPlot[:, :] = np.nan
trainPredictPlot[begin:end, :] = trainPredict

trainTruePlot = np.empty_like(data, dtype=float)  # Ensure dtype is float
trainTruePlot[:, :] = np.nan
trainTruePlot[begin:end, :] = data[look_back:len(trainY)+look_back]

# Plot baseline and predictions
plt.figure(figsize=(24, 6))  # Adjusted figure size
#plt.plot(data_org, label='Original Data')
plt.plot(trainYPlot, label='Cleaned Values', color='blue')
plt.plot(trainTruePlot, label='True Values', color='green')
plt.plot(trainPredictPlot, label='Predicted Values', color='red')

plt.legend()
plt.show()

In [ ]:
begin = train_size + look_back
end = begin + len(testPredict)

testYPlot = np.empty_like(data, dtype=float)  # Ensure dtype is float
testYPlot[:, :] = np.nan
testYPlot[begin:end, :] = testY

testPredictPlot = np.empty_like(data, dtype=float)  # Ensure dtype is float
testPredictPlot[:, :] = np.nan
testPredictPlot[begin:end, :] = testPredict

testTruePlot = np.empty_like(data, dtype=float)  # Ensure dtype is float
testTruePlot[:, :] = np.nan
testTruePlot[begin:end, :] = data[-len(testY):]

# Plot baseline and predictions
plt.figure(figsize=(24, 6))  # Adjusted figure size
#plt.plot(data_org, label='Original Data')
plt.plot(testYPlot, label='Cleaned Values', color='blue')
plt.plot(testTruePlot, label='True Values', color='green')
plt.plot(testPredictPlot, label='Predicted Values', color='red')


plt.legend()  
plt.show()